# BIR Revenue Regulation Scraper

## Information
- Gets Revenue Issuances from `https://www.bir.gov.ph/revenue-issuances-details` for each year
- Puts each file in a folder corresponding to each year

## Code

### Dependency Installation

In [1]:
!uv pip install --requirements requirements.txt
!playwright install

Audited 45 packages in 6ms


### Imports

In [20]:
from playwright.async_api import async_playwright, Page, Browser
import asyncio
import os
import json
import requests
MAX_CONCURRENCY = 5
semaphore = asyncio.Semaphore(MAX_CONCURRENCY)

### Scraper Code

#### File Class Definition

In [3]:
class File:
    def __init__(self, issuance_number, subject_matter, date_of_issue, is_file_downloadable):
        self.issuance_number = issuance_number
        self.subject_matter = subject_matter
        self.date_of_issue = date_of_issue
        self.is_file_downloadable = is_file_downloadable
items : list[File] = []

def json_encoder(obj: File):
    if isinstance(obj, File):
        return {
            'issuance_number' : obj.issuance_number,
            'subject_matter' : obj.subject_matter,
            'date_of_issue' : obj.date_of_issue,
            'is_file_downloadable' : obj.is_file_downloadable
        }
    raise TypeError("Object is not JSON parsable.")

#### Helper Functions

##### Download File Function


Downloads a file given the URL and the destination directory path, with the ability to create the directory if it does not exist

In [12]:
def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)
    print(f"URL: {url}")
    r = requests.get(url, stream=True)
    if r.ok:
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        return False
    return True




In [18]:
async def go_through_nonarchive_revenue_issuance_type(rr: Locator, page: Page):
    """
    (SEQUENTIAL) Gathers all required data (URL and Type) from a single RR card.
    Returns a list of tuples: [(url, type), (url, type), ...]
    """
    data_for_parallel_tasks = []
    
    # 1. Get revenue issuance type
    text_div_locators = await rr.locator("div.richtext.prose p").all()
    if not text_div_locators:
        print("Warning: Could not find rich text div.")
        return data_for_parallel_tasks

    revenue_issuance_type_text = await text_div_locators[0].inner_text(timeout=90000)
    # Use strip() for robust cleaning
    revenue_issuance_type = revenue_issuance_type_text.split(' (')[0].strip() 
    # print(f"Gathering links for: {revenue_issuance_type}")
    
    # 2. Get all year links (non-archive)
    if len(text_div_locators) > 1:
        link_locator_list = await text_div_locators[-1].locator("a").all()
        # 3. Extract the href attribute (the URL string)
        for link_locator in link_locator_list[:5]: # Use [:5] limit as in original code
            url = await link_locator.get_attribute('href')
            if url:
                data_for_parallel_tasks.append((url, revenue_issuance_type))
                
    return data_for_parallel_tasks

async def go_through_archive(page : Page):
    """
    (SEQUENTIAL) Gathers all required data (URL and Type) from the RR archive.
    Returns a list of tuples: [(url, type), (url, type), ...]
    """
    data_for_parallel_tasks = []
    archive_dropdown_div_list = await page.locator("#main .bg-white .container .border").all()
    for dropdown_div in archive_dropdown_div_list:
        revenue_issuance_type = await dropdown_div.locator('.leading-6').first.inner_text(timeout=90000)
        # print(f"Gathering links for: {revenue_issuance_type}")
        await dropdown_div.click()
        await page.wait_for_selector("#main .bg-white .container .border .richtext.prose", timeout=6000)
        for link in await dropdown_div.locator("p a").all():
            url = await link.get_attribute('href')
            data_for_parallel_tasks.append((url, revenue_issuance_type))
    return data_for_parallel_tasks
    

async def go_through_file_table_PARALLEL(browser: Browser, url: str, revenue_issuance_type: str):
    """
    Handles a single scraping task by creating an isolated page and navigating directly.
    """
    # 1. Create new isolated context and page for this task
    context = await browser.new_context(viewport={"width": 1280, "height": 720})
    new_page = await context.new_page()

    try:
        # 2. Navigate directly to the URL passed from the discovery phase
        print(f"[{revenue_issuance_type}] Navigating to: {url}")
        await new_page.goto(url, timeout=120000, wait_until="domcontentloaded")
        # Wait for main content to load. Using a reliable selector like 'tbody tr' visibility is key.
        await new_page.wait_for_selector("tbody tr", state='visible', timeout=120000)
        
        # Print Current Page
        heading_locator = new_page.locator('#main .bg-white .container .text-2xl')
        heading_text = await heading_locator.first.inner_text(timeout=90000)
        selected_year = heading_text.split(' ')[0]
        
        print(f"[{revenue_issuance_type}] Getting files for {selected_year}...")

        file_items = await new_page.locator("tbody tr").all()
        
        for file_item in file_items:
            issuance_number = await file_item.locator('td').nth(0).inner_text(timeout=90000)
            subject_matter = await file_item.locator('td').nth(1).inner_text(timeout=90000)
            date_of_issue = await file_item.locator('td').nth(2).inner_text(timeout=90000)
            # Use .first to resolve strict mode violation if multiple "Full Text" links exist
            download_link_locator = file_item.locator('a:has-text("Full Text")').first

            file_download_link = await download_link_locator.get_attribute('href')
            
            # Call the synchronous download function
            download_result = download(file_download_link, f"BIR_Revenue_Files/{revenue_issuance_type}/{selected_year}/")
            class_item = File(issuance_number, subject_matter, date_of_issue, download_result)
            items.append(class_item)
            
            
    except Exception as e:
        print(f"[{revenue_issuance_type}] ERROR scraping {url}: {e}")
    finally:
        # 3. Ensure the isolated context is closed
        await context.close()

#### Scraper Implementation

In [19]:
# Launch browser only once
async with async_playwright() as pw:
    browser = await pw.chromium.launch(headless=False) # Set headless=True for faster execution
    context = await browser.new_context(viewport={"width": 1280, "height": 720})
    page = await context.new_page()

    print("--- Phase 1: Discovery (Sequential) ---")
    
    # go to the page and wait for the content to load
    await page.goto("https://www.bir.gov.ph/revenue-issuances-details")
    await page.wait_for_selector("div.cursor-pointer", state='visible', timeout=60000)
    await page.wait_for_timeout(200)
    # get all revenue issuance type card locators
    revenue_issuance_type_locators = await page.locator("div.cursor-pointer").all()
    print(len(revenue_issuance_type_locators))
    # Create sequential tasks to gather data from the main page
    discovery_tasks = [
        go_through_nonarchive_revenue_issuance_type(rr, page)
        for rr in revenue_issuance_type_locators
    ]

    # Run discovery tasks (they only READ from the page)
    nonarchive_data_lists = await asyncio.gather(*discovery_tasks)

    await page.goto("https://www.bir.gov.ph/bir-archive?type=EXTERNAL%20LINK&external_url=https://www.bir.gov.ph/bir-archive&label=Revenue%20Regulations%20Previous%20Years", wait_until='domcontentloaded')
    await page.wait_for_selector("#main .bg-white .container .border", state='visible', timeout=60000)
    await page.wait_for_timeout(200)
    archive_urls_to_scrape_concurrently = await go_through_archive(page)
    # Flatten the list of lists into a list of tuples: [(url, type), ...]
    nonarchive_urls_to_scrape_concurrently = [
        item for sublist in (nonarchive_data_lists) for item in sublist
    ]
    # Close the initial context/page used only for discovery
    await context.close() 

    
    print(f"\n--- Phase 2: Scraping {len(nonarchive_urls_to_scrape_concurrently) + len(archive_urls_to_scrape_concurrently)} Links (Parallel) ---")
    

    # Create the parallel scraping tasks
    urls_to_scrape = nonarchive_urls_to_scrape_concurrently + archive_urls_to_scrape_concurrently

    for url, rr_type in urls_to_scrape:    
        await go_through_file_table_PARALLEL(browser, f"https://www.bir.gov.ph/{url}", rr_type)
    # Run all scraping tasks concurrently

    print("\n--- Scraping Complete ---")
    await browser.close()

--- Phase 1: Discovery (Sequential) ---
5

--- Phase 2: Scraping 115 Links (Parallel) ---
[Revenue Regulations] Navigating to: https://www.bir.gov.ph/2025-Revenue-Regulations?type=PAGE&to=2025-Revenue-Regulations&label=2025 Revenue Regulations
[Revenue Regulations] Getting files for 2025...
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR NO. 27-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 26-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 25-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 24-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 23-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 22-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR NO. 21-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR NO. 20-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR NO. 19-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR NO. 18-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 17-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RR No. 16-2025.pdf


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first



URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2016-2023%20FINAL.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2015-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2014-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2013-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2012-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2011-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%2010-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%209-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%208-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%207-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%206-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%205-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%204-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%203-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMO%20No.%202-2023.pdf
URL: https

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first



URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2050-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2049-2023_copy.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2048-2023%20(1).pdf


...

URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2047-2023.pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2046-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2045-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2044-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2043-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2042-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2041-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2040-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2039-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2038-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2037-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2036-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2035-2023%20(1).pdf
URL: https://bir-cdn.bir.gov.ph/local/pdf/RMC%20No.%2034-2023%20(1).pd

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 73-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 72-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 71-2025 corected.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 70-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 69-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 68-2025.pdf


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("tbody tr").first.locator(".richtext.prose p").first



URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 67-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 66-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 65-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 64-2025 (1).pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 63-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 62-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 61-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 60-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 59-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 58-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 57-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 56-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 55-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 54-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO No. 53-2025.pdf
URL: https://bir-cdn.bir.gov.ph/BIR/pdf/RDAO NO. 52-2025.pdf
URL: https://bir-cdn

In [25]:
with open('BIR_Revenue_Files/json_output.json', mode='w', encoding='utf-8') as f:
    json.dump(
        obj=items,
        fp=f,
        default=json_encoder,
        indent=4
    )